In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_image_path = glob.glob('/home/jhr/Documents/data/dc_2000/train/*/*.jpg')

In [ ]:
len(train_image_path)

In [ ]:
train_image_path[:5]

In [ ]:
train_image_path[-5:]

用python的切片测试label编码

In [ ]:
p = '/home/jhr/Documents/data/dc_2000/train/dog/dog.524.jpg'

In [ ]:
int(p.split('/')[-1].split('.')[0] == 'dog')

利用切片进行labels编码

In [ ]:
train_image_label = [int(p.split('/')[-1].split('.')[0] == 'cat')
                     for p in train_image_path]

train_image_label[:5]

In [ ]:
def load_precess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    image = tf.cast(image, tf.float32)
    image = image/255  # 归一化
    label = tf.reshape(label, [1])
    return image, label

# tf.image.convert_image_dtype#该函数对于图片是32的会自动做归一化处理

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
    (train_image_path, train_image_label))
AUTOTUNE = tf.data.experimental.AUTOTUNE
dataset = dataset.map(load_precess_image,
                      num_parallel_calls=AUTOTUNE)

In [ ]:
dataset

In [ ]:
BATCH_SIZE = 32
train_count = len(train_image_path)

In [ ]:
# 对数据进行batch，shuffle预处理
dataset = dataset.shuffle(train_count).batch(BATCH_SIZE)
dataset = dataset.prefetch(AUTOTUNE)  # 后台读取数据

In [ ]:
# model = keras.Sequential([
#     tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
#         256, 256, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),  # 图像缩小1倍
#     tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),  # 缩小
#     tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),  # 缩小
#     tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),  # 缩小
#     tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(1)
# ])

## 优化模型1

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
        256, 256, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
        256, 256, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # 图像缩小1倍
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # 缩小
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # 缩小
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # 缩小
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)  # 注意，这里没有选用优化函数，softmoix
])

## 优化模型2

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
        256, 256, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
        256, 256, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.summary()

In [ ]:
# 在dataset中取出一个step的数据
imgs, labels = next(iter(dataset))
imgs.shape

In [ ]:
plt.imshow(imgs[0])

In [ ]:
# 对数据进行一个step的计算
pred = model(imgs)

In [ ]:
pred.shape

In [ ]:
np.array([p[0].numpy() for p in tf.cast(pred > 0, tf.int32)])

In [ ]:
np.array([l[0].numpy() for l in tf.cast(labels > 0, tf.int32)])

In [ ]:
# 损失函数
ls = tf.keras.losses.BinaryCrossentropy()
ls([0., 0., 1., 1.], [1., 1., 1., 1.])

In [ ]:
optimizer = tf.keras.optimizers.Adam()
epoch_loss_avg = tf.keras.metrics.Mean('train_loss')  # 计算损失值
train_accuracy = tf.keras.metrics.Accuracy('train_acc')  # 计算正确率

epoch_loss_avg_test = tf.keras.metrics.Mean('train_loss_test')  # 计算损失值
train_accuracy_test = tf.keras.metrics.Accuracy('train_acc_test')  # 计算正确率

In [ ]:
# 利用损失函数，计算一次的准确率
a = train_accuracy(labels, tf.cast(pred > 0, tf.int32))
a

In [ ]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        # 模型中没有激活函数，from_logis设置为true，进行内部激活，
        # 该函数计算损失值
        loss_step = tf.keras.losses.BinaryCrossentropy(
            from_logits=True)(labels, pred)
    # 计算梯度
    grads = t.gradient(loss_step, model.trainable_variables)
    # 优化参数
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    epoch_loss_avg(loss_step)
    train_accuracy(labels, tf.cast(pred > 0, tf.int32))  # 计算准确率

In [ ]:
def test_step(model, images, labels):
    pred = model.predict(images)  # 预测模式
    loss_step = tf.keras.losses.BinaryCrossentropy(
        from_logits=True)(labels, pred)
    epoch_loss_avg_test(loss_step)
    train_accuracy_test(labels, tf.cast(pred > 0, tf.int32))  # 计算准确率

In [ ]:
train_loss_result = []
train_acc_result = []
train_loss_result_test = []
train_acc_result_test = []
num_epochs = 5
for epoch in range(num_epochs):
    for imas_, labels_ in dataset:
        train_step(model, imas_, labels_)
        print(".", end='')
    print()
    train_loss_result.append(epoch_loss_avg.result())
    train_acc_result.append(train_accuracy.result())
    print("epoch:{},loss:{:.3f},acc:{:.3f}".format(epoch+1, epoch_loss_avg.result(),
                                                   train_accuracy.result()))

    for imas_, labels_ in dataset:
        test_step(model, imas_, labels_)
        print(".", end='')
    print()
    train_loss_result_test.append(epoch_loss_avg_test.result())
    train_acc_result_test.append(train_accuracy_test.result())
    print("testing,loss:{:.3f},acc:{:.3f}".format(epoch_loss_avg_test.result(),
                                                  train_accuracy_test.result()))
    epoch_loss_avg.reset_states()
    train_accuracy.reset_states()
    epoch_loss_avg_test.reset_states()
    train_accuracy_test.reset_states()

In [ ]:
plt.plot(range(1,epoch+2),train_loss_result,label="train_loss_result")
plt.plot(range(1,epoch+2),train_acc_result,label="train_acc_result")
plt.legend()

# 猫狗数据增强

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_image_path = glob.glob('/home/jhr/Documents/data/dc_2000/train/*/*.jpg')
train_image_label = [int(p.split('/')[-1].split('.')[0] == 'cat')
                     for p in train_image_path]
train_image_label[:5],len(train_image_path)

In [ ]:
train_image_path_test = glob.glob('/home/jhr/Documents/data/dc_2000/test/*/*.jpg')
train_image_label_test  = [int(p.split('/')[-1].split('.')[0] == 'cat')
                     for p in train_image_path_test ]

train_image_label_test [:5],len(train_image_path_test)

In [ ]:
#图片增强
def load_precess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    # image = tf.image.random_crop(image, [256, 256, 3])  # 随机裁剪
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # image = tf.image.random_brightness(image, 0.5)
    # image = tf.image.random_hue(image,0.5)
    # image = tf.image.random_contrast(image, 0, 1)
    image = tf.cast(image, tf.float32)
    image = image/255  # 归一化
    label = tf.reshape(label, [1])
    return image, label

def load_precess_image_test(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    image = tf.cast(image, tf.float32)
    image = image/255  # 归一化
    label = tf.reshape(label, [1])
    return image, label

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
    (train_image_path, train_image_label))
AUTOTUNE = tf.data.experimental.AUTOTUNE
dataset = dataset.map(load_precess_image,
                      num_parallel_calls=AUTOTUNE)

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices(
    (train_image_path_test, train_image_label_test))
dataset_test = dataset_test.map(load_precess_image_test,
                      num_parallel_calls=AUTOTUNE)

In [ ]:
# for img,label in dataset.take(1):
#     plt.imshow(img)

In [ ]:
# for img_test,label_test in dataset_test.take(1):
#     plt.imshow(img_test)

In [ ]:
step=32
dataset=dataset.shuffle(2000).repeat().batch(step)
dataset_test=dataset_test.batch(step)

In [ ]:
dataset

In [ ]:
dataset_test

## VGG网络

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(256,(1,1),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(1,1),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(1,1),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.GlobalAveragePooling2D())#全连接层 
model.add(tf.keras.layers.Dense(4096,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(4096,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1000,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3)))#卷积层
# model.add(tf.keras.layers.BatchNormalization())#批标准化层
# model.add(tf.keras.layers.Conv2D(64,(3,3)))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D())#连接层

# model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D())#连接层

# model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D())

# model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D())

# model.add(tf.keras.layers.Conv2D(1024,(3,3),activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.GlobalAveragePooling2D())
# model.add(tf.keras.layers.Dense(1024,activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dense(256,activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),activation='relu'))
# model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
# model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
# model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(1024,(3,3),activation='relu'))
# model.add(tf.keras.layers.GlobalAveragePooling2D())#全局平均池化
# model.add(tf.keras.layers.Dense(1024,activation='relu'))
# model.add(tf.keras.layers.Dense(256,activation='relu'))
# model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
             )

In [ ]:
history=model.fit(dataset,
          epochs=100,
          steps_per_epoch=2000//step,
          validation_data=dataset_test,
          validation_steps=1000//step
         )

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.epoch,history.history.get('accuracy'),label='acc')
plt.plot(history.epoch,history.history.get('val_accuracy'),label='val_acc')

In [ ]:
plt.plot(history.epoch,history.history.get('loss'),label='loss')
plt.plot(history.epoch,history.history.get('val_loss'),label='val_loss')